In [ ]:
!pip install ipykernel transformer_lens plotly 

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=7

In [ ]:
import transformer_lens
import torch
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
import numpy as np

from jaxtyping import Float
from functools import partial
import numpy as np

# Convert HF Transformers weights to HookedTransformer weights

In [ ]:
"""
from transformer_lens.pretrained.weight_conversions.llama import convert_llama_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_config(config)
model.load_state_dict(torch.load("checkpoints/llama_instruct_value_assign_finetuned_new.weights", weights_only=True))
hooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
new_state_dict = convert_llama_weights(model, hooked_config)
torch.save(new_state_dict, "checkpoints/llama_instruct_value_assign_finetuned_hooked_new.weights")

"""

In [ ]:
config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
model = transformer_lens.HookedTransformer(config)
model.load_and_process_state_dict(torch.load("checkpoints/llama_instruct_value_assign_finetuned_hooked_new.weights", weights_only=True))
#config = transformer_lens.loading.get_pretrained_model_config("Qwen/Qwen2.5-1.5B-Instruct")
#model = transformer_lens.HookedTransformer(config)
#model.load_and_process_state_dict(torch.load("checkpoints/Qwen2.5-1.5B-Instruct_value_assign_finetuned_hooked.weights", weights_only=True))


In [ ]:
print(model)

In [ ]:
charset_from = ""
for id in range(0, 50000):
    token = model.tokenizer.decode(id, skip_special_tokens=True)
    if len(token) > 1 or token == "�":
        continue
    decode_id = model.tokenizer.encode(token, add_special_tokens=False)
    if len(decode_id) == 1 and decode_id[0] == id:
        charset_from += token

In [ ]:
charset_from[:105]

In [ ]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../generators")
sys.path.insert(0, "generators")


from generators.value_assignment import ValueAssignmentGenerator
import transformers
import itertools


data_generator = ValueAssignmentGenerator(tokenizer=model.tokenizer,
                                       seed=0,
                                       #use_few_shot=True, 
                                       #use_instruction=True, 
                                       #apply_chat_template=True,
                                       length=(3, 3),
                                       encoding_length=(3, 3),
                                       )
samples = itertools.islice(data_generator.generate_samples(), 30)

In [ ]:
samples = list(samples)
texts = [sample[0] for sample in samples]
input_ids = [sample[1] for sample in samples]
target_ids = [sample[2] for sample in samples]
attn_labels = [sample[3] for sample in samples]

In [ ]:
texts[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import plotly.graph_objects as go


def visualize_attention_heads(attention_matrix, tokens=None, title="Attention Heatmap", save_path=None, heads=None, length=None, attn_labels=None):
    """
    Visualize a multi-head attention matrix as heatmaps for each head.

    Parameters:
        attention_matrix (torch.Tensor): Tensor of shape (1, num_heads, seq_len, seq_len).
        tokens (list, optional): List of tokens corresponding to the rows/columns of the matrix.
        title (str): Title of the heatmap.
        save_path (str, optional): Path to save the heatmap images. If None, the heatmaps are not saved.
        heads (list, optional): List of head indices to visualize.
        length (int, optional): Number of tokens to display from the lower-right corner.
"""
    scores = attention_matrix.squeeze(0).cpu().detach().numpy()  # Shape: (num_heads, seq_len, seq_len)
    num_heads = scores.shape[0]
    seq_len = scores.shape[1]

    for head in range(num_heads):
        if heads is not None and head not in heads:
            continue
        head_scores = scores[head]  # Shape: (seq_len, seq_len)

        # Apply filtering for the lower-right "length x length" square
        if length is not None and length < seq_len:
            head_scores = head_scores[-length:, -length:]
            if tokens:
                tokens = tokens[-length:]
            if attn_labels is not None:
                attn_labels = attn_labels[-length:, -length:]

        # Normalize scores for better visualization
        min_score = head_scores.min()
        max_score = head_scores.max()
        normalized = (head_scores - min_score) / (max_score - min_score)  # Normalize to [0, 1]

        # Create the heatmap
        fig = go.Figure(
            data=go.Heatmap(
                z=normalized,
                x=tokens if tokens else list(range(len(normalized))),
                y=tokens if tokens else list(range(len(normalized))),
                colorscale="Viridis",
                colorbar=dict(title="Attention Score"),
            )
        )

        # Add title and layout
        fig.update_layout(
            title=f"{title} - Head {head + 1}",
            xaxis=dict(title="Tokens", tickangle=45),
            yaxis=dict(title="Tokens"),
        )

        # Highlight attention label regions if provided
        if attn_labels is not None:
            for i in range(len(attn_labels)):
                for j in range(len(attn_labels)):
                    if attn_labels[i, j] == 1:
                        fig.add_shape(
                            type="rect",
                            x0=j - 0.5,
                            x1=j + 0.5,
                            y0=i - 0.5,
                            y1=i + 0.5,
                            line=dict(color="red", width=2),
                        )

        # Save the plot if a save path is provided
        if save_path:
            fig.write_image(f"{save_path}_head_{head + 1}.png")

        # Show the plot
        fig.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

mask = None

def compute_scores(logits, target):
    ignore_index = -100  # Optional: index to ignore in loss/accuracy computation
    # Compute loss
    loss = torch.nn.functional.cross_entropy(
    logits.view(-1, logits.size(-1)),  # Flatten logits to (batch_size * seq_len, vocab_size)
    target.view(-1),               # Flatten target_ids to (batch_size * seq_len)
    ignore_index=ignore_index          # Ignore padding tokens if applicable
)

    # Compute accuracy
    predictions = torch.argmax(logits, dim=-1)  # Get predicted token indices
    correct = (predictions == target) & (target != ignore_index)  # Ignore padding tokens
    accuracy = correct.sum().item() / (target != ignore_index).sum().item()

    print(f"Loss: {loss.item()}, Accuracy: {accuracy:.2%}")

    errors = (predictions != target) & (target != ignore_index)  # Identify error positions
    errors_np = errors.cpu().numpy()  # Convert to NumPy for visualization
    #print("Errors:", errors_np)

    return loss.item(), accuracy

def randomly_expand_mask(mask, num_additional_trues):
    """
    Randomly add more `True` values to a boolean mask in the lower triangle.

    Args:
        mask (torch.Tensor): A boolean tensor of shape (seq_len, seq_len) with `True` values.
        num_additional_trues (int): Number of additional `True` values to add.

    Returns:
        torch.Tensor: The expanded mask with more `True` values.
    """
    seq_len = mask.shape[0]

    # Ensure the mask is in the lower triangle
    causal_mask = torch.tril(torch.ones((seq_len, seq_len), dtype=torch.bool, device=mask.device))

    # Find positions in the lower triangle that are currently `False`
    available_positions = (~mask) & causal_mask
    available_indices = torch.nonzero(available_positions, as_tuple=False)  # Shape: (num_available, 2)

    # Randomly select a subset of these positions
    if available_indices.size(0) > 0:
        num_to_add = min(num_additional_trues, available_indices.size(0))
        selected_indices = available_indices[torch.randperm(available_indices.size(0))[:num_to_add]]

        # Set the selected positions to `True`
        mask[selected_indices[:, 0], selected_indices[:, 1]] = True

    return mask

def fix_attn_pattern_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    attn_labels_1_idx,
    heads_to_reinforce,
    attn_labels,
    keep_last_n_tokens,
    threshold=3e-3,
    expand=10
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    global mask
    batch, heads, seq_len, _ = value.shape

    # Create a mask initialized to False
    mask = torch.zeros((seq_len, seq_len), dtype=torch.bool, device=value.device)

    # Set diagonal positions to True
    mask |= torch.eye(seq_len, dtype=torch.bool, device=value.device)

    # Set the first token (column 0) to True
    mask[:, 0] = True

    # Set positions in attn_labels_1_idx to True
    for x, y in attn_labels_1_idx:
        mask[x, y] = True

    # Expand the mask to match the shape of attention_scores
    mask = mask.unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, seq_len, seq_len)
    mask = mask.expand(batch, heads, seq_len, seq_len)  # Shape: (batch, heads, seq_len, seq_len)
    print(mask)
    mask = randomly_expand_mask(mask, expand)
    print(mask)
    # Set positions not in the mask to -inf
    value = value.masked_fill(~mask, float('-inf'))

    # Compute the softmax probabilities after adjustment
    softmax_scores_after = torch.softmax(value, dim=-1)
    """
    # Verify that the probabilities of sink, diagonal, and reference tokens remain unchanged
    for b in range(batch):
        for h in range(heads):
            for i in range(seq_len):
                # Sink token (first column)
                assert torch.isclose(
                    softmax_scores_before[b, h, i, 0], softmax_scores_after[b, h, i, 0], atol=5e-1
                ), f"Sink token probability mismatch at batch {b}, head {h}, row {i}. Before: {softmax_scores_before[b, h, i, 0]} After: {softmax_scores_after[b, h, i, 0]}"

                # Diagonal token
                assert torch.isclose(
                    softmax_scores_before[b, h, i, i], softmax_scores_after[b, h, i, i], atol=5e-1
                ), f"Diagonal token probability mismatch at batch {b}, head {h}, row {i}. Before: {softmax_scores_before[b, h, i, 0]} After: {softmax_scores_after[b, h, i, 0]}"

                # Reference tokens
                for x, y in attn_labels_1_idx:
                    if x == i:  # Same row
                        assert torch.isclose(
                            softmax_scores_before[b, h, x, y], softmax_scores_after[b, h, x, y], atol=5e-1
                        ), f"Reference token probability mismatch at batch {b}, head {h}, row {x}, col {y}. Before: {softmax_scores_before[b, h, i, 0]} After: {softmax_scores_after[b, h, i, 0]}"

    """
    return value

def vis_attn_pattern_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    attn_labels_1_idx,
    heads_to_reinforce,
    attn_labels,
    keep_last_n_tokens,
    threshold=3e-3
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    batch, heads, seq_len, _ = value.shape 
    layer = int(hook.name.split(".")[1])
    if layer not in heads_to_reinforce.keys():
        return
    
    for h in range(heads):
        if h not in heads_to_reinforce[layer]:
            continue
        acc = 0
        
        for i in range(seq_len):
            for j in range(seq_len):
                if (i, j) in attn_labels_1_idx and value[0, h, i, j] >= threshold:
                    value[0, h, i, j] += 1
                    #acc += value[0, h, i, j]
                    pass
        #best_heads[(layer, h)] = best_heads.get((layer, h), 0) + acc
        #visualize_attention_heads(value, None, title=f"Example Attention Heatmap Layer {layer}", heads=[h], length=keep_last_n_tokens, attn_labels=attn_labels)

    return value
    pass

device = "cuda"

is_attn_scores = lambda name: name.endswith("hook_attn_scores")
is_attn_pattern = lambda name: name.endswith("hook_pattern")
ablated = []
for idx in range(0, 30):
    
    indices =  np.argwhere(attn_labels[idx].cpu().numpy() == 1)
    attn_labels_1_idx = set(map(tuple, indices))
    

    model.to(device)
    input_ids[idx] = input_ids[idx].to(device)
    target_ids[idx] = target_ids[idx].to(device)
    attn_labels[idx] = attn_labels[idx].to(device)
                

    heads_to_reinforce_x=[
        (7, [5]),
        (8, [20, 31]),
        (9, [8, 10]),
        (10, [23, 20, 21, 1, 22]),
        (11, [14, 20, 4]),
        (12, [13, 2, 29]),
        (13, [20, 21, 22, 2, 23, 31, 1, 11]),
        (14, [22, 11, 20, 15, 21, 14, 31, 3, 28, 29]),
        (15, [31, 7, 22, 30, 20, 29]),
    ]
    threshold = 0
    cols_to_keep = ~np.all(attn_labels[idx].cpu().numpy() == -100, axis=0)
    keep_last_n_tokens = 400
    heads_x = dict(heads_to_reinforce_x)
    logits = model.run_with_hooks(input_ids[idx], return_type="logits",
                                    fwd_hooks=[
                                        (is_attn_scores, partial(fix_attn_pattern_hook, attn_labels_1_idx=attn_labels_1_idx, heads_to_reinforce=heads_x, attn_labels=attn_labels[idx], keep_last_n_tokens = keep_last_n_tokens, expand=5)),
                                        (is_attn_pattern, partial(vis_attn_pattern_hook, attn_labels_1_idx=attn_labels_1_idx, heads_to_reinforce=heads_x, attn_labels=attn_labels[idx], keep_last_n_tokens = keep_last_n_tokens))
                                    
                                    ])
    loss, acc = compute_scores(logits, target_ids[idx])
    ablated.append((heads_x, loss, acc))
    del logits
    torch.cuda.empty_cache()
    

In [ ]:
mask

In [ ]:
sums = 0
for item in ablated:
    sums += item[2]
print(sums/len(ablated))
print(sums)

In [ ]:
for item in sorted(best_heads.items(), key=lambda item: item[1], reverse=True):
    print(item[0][0], item[0][1], item[1].item())